# App tests

Test the Solar Flask app.

## App code (clean, to copy in .py)

In [1]:
import os
import sys
import flask
import requests
import skimage
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Directory of the mrcnn library
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN and solar.py
sys.path.append(ROOT_DIR)
from mrcnn import visualize
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from samples.solar import solar

# Path to trained weights file
MODEL_NAME = "mask_rcnn_solar_0025.h5"
SOLAR_WEIGHTS_PATH = os.path.join(ROOT_DIR, "logs", MODEL_NAME)

# Path to results directory (TBD)
RESULTS_DIR = os.path.join(ROOT_DIR, "results")
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

Using TensorFlow backend.


In [3]:
# Check the correct versions of tensorflow (1.15.2) and keras (2.1.0) have been loaded
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

1.15.2
2.1.0


In [4]:
app = flask.Flask(__name__)

In [5]:
@app.route('/', methods=["POST", "GET"])
def home():
    """ Simple view function run when the route specified above 
        is requested by user. Displays home page based on html template.
    """
    return flask.render_template("index.html")

In [6]:
UPLOAD_FOLDER = 'static'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

In [ ]:
@app.route('/success', methods=["POST", "GET"])
def predict():

    # Check if the post request has either an uploaded or default image
    if request.method == 'POST':
        
        if flask.request.files['file']:
            image = flask.request.files['file']
            image.save(os.path.join(app.config['UPLOAD_FOLDER'], 'upload.png'))
            # If we save the file under its original filename, we should
            # run it through secure_filename for security before saving,
            # with: filename = secure_filename(image.filename)
        
        elif flask.request.form.get('default_image'):
            image = flask.request.form.get('default_image', None)
            if image is None:
                flask.flash('No file uploaded')
                return redirect(request.url)
            
        # Read the image as a numpy array and resize it
        image = skimage.io.imread(image)
        image, window, scale, padding, crop = utils.resize_image(image,
                                                min_dim=inference_config.IMAGE_MIN_DIM,
                                                min_scale=inference_config.IMAGE_MIN_SCALE,
                                                max_dim=inference_config.IMAGE_MAX_DIM,
                                                mode=inference_config.IMAGE_RESIZE_MODE)

        # Run detection
        yhat = model.detect([image], verbose=0)[0]
        n_solar = yhat['masks'].shape[2]
        print("Nb of solar arrays detected: ", n_solar)
        pv_surface = compute_mask_to_surface(yhat['masks'])
        pv_size_annotation = [str(round(x,1))+"m^2" for x in pv_surface]

        # Save prediction in static folder
        visualize.display_instances(image, yhat['rois'], yhat['masks'],
                                    yhat['class_ids'], class_names=['BG', 'solar array'],
                                    yhat['scores'], captions=pv_size_annotation,
                                    title="{} PV arrays detected".format(n_solar),
                                    plot=False, save="static/result.png")
        
        # Save mask in static folder
        mask = yhat['masks'].astype(int)
        mask = mask[...,0:1]+mask[...,1:2]
        mask = mask.squeeze()
        skimage.io.imsave("static/mask.png", mask)
   
        # Save detection information in static folder
        data = {}
        data["Successful detection"] = True
        data["Number of detected solar arrays"] = str(n_solar)
        data["Total surface of detected solar arrays"] = "≈"+str(round(pv_surface.sum(),1))+"m^2"
        data["Detection confidence level"] = "≈"+str(round(yhat['scores'].mean()*100,1))+"%"
        # Save as json
        # with open('static/detection_info.json', 'w') as outfile:
        #    json.dump(data, outfile)
        # Save as Excel (for non-tech people)
        df = pd.DataFrame(data, index=[0]).T
        df.to_excel('static/detection_info.xlsx', header=False)
        
        return redirect(url_for('upload')) # not sure if we need this
    
    return render_template('success_nst.html')

In [17]:
if __name__ == '__main__':
# the app will run only if it's called as main,
# i.e. not if you import the app in another code
    print("...Loading model and starting server\n",
          "...please wait until server has fully started")
     
    # Create model in inference mode
    inference_config = solar.InferenceConfig()
    
    # Recreate the model in inference mode
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=SOLAR_WEIGHTS_PATH)
    # Load trained weights
    model.load_weights(SOLAR_WEIGHTS_PATH, by_name=True)
    print("Loaded model:", MODEL_NAME)
    
    # Run app
    app.run(host='0.0.0.0', port='5000', debug=True, use_reloader=False)

...Loading model and starting server
...    please wait until server has fully started
Loaded model: /Users/Thomas/Documents/Data Science X/Stage/Total/Solar Panels/Githubs/Mask_RCNN/logs/mask_rcnn_solar_0025.h5
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


## Testing

The `dataset.load_image()` function of Mask-RCNN takes as argument an `image_id`, which means the image has to be part of a dataset organized and loaded as a Mask-RCNN `Dataset` instance. To detect solar panels in a single image uploaded by the user, we just use simple skimage functions as part of the script.

In [8]:
# Create model in inference mode
inference_config = solar.InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=SOLAR_WEIGHTS_PATH)
# Load trained weights
model.load_weights(SOLAR_WEIGHTS_PATH, by_name=True)
print("Loaded model:", MODEL_NAME)






Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.






Loaded model: mask_rcnn_solar_0025.h5


In [11]:
# We read the image as a numpy array and resize it
#from skimage import io

image = skimage.io.imread("/Users/Thomas/Documents/Data Science X/Stage/Total/Solar Panels/Githubs/Mask_RCNN/datasets/solar/images_stockton/10sfg465970_09_09.png")
image, window, scale, padding, crop = utils.resize_image(
                                        image,
                                        min_dim=inference_config.IMAGE_MIN_DIM,
                                        min_scale=inference_config.IMAGE_MIN_SCALE,
                                        max_dim=inference_config.IMAGE_MAX_DIM,
                                        mode=inference_config.IMAGE_RESIZE_MODE)            

# We run detection
yhat = model.detect([image], verbose=0)[0]
n_solar = yhat['masks'].shape[2]
print("Nb of solar arrays detected: ", n_solar)

Nb of solar arrays detected:  2


In [67]:
visualize.display_instances(image, yhat['rois'], yhat['masks'],
                            yhat['class_ids'], ['BG', 'solar array'],
                            yhat['scores'], plot=False, save="static/result.png")

In [37]:
mask = yhat['masks'].astype(int)
mask = mask[...,0:1]+mask[...,1:2]
mask = mask.squeeze()
skimage.io.imsave("static/mask.png", mask)

In [59]:
# Save detection information in static folder
data={}
pv_surface = solar.compute_mask_to_surface(yhat['masks'])
import json

data["success"] = True
data["Number of detected solar arrays"] = str(n_solar)
data["Total surface of detected solar arrays"] = "≈"+str(round(pv_surface.sum(),1))+"m^2"
data["Detection confidence level"] = "≈"+str(round(yhat['scores'].mean()*100,1))+"%"

with open('static/detection_info.json', 'w') as outfile:
    json.dump(data, outfile)

In [64]:
import pandas as pd
df = pd.DataFrame(data, index=[0]).T

print (df)

df.to_excel('static/detection_info.xlsx', header=False)

                                              0
success                                    True
Number of detected solar arrays               2
Total surface of detected solar arrays  ≈3.0m^2
Detection confidence level               ≈97.5%
